# Import library and package

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# konfigurasi
input_size = 784 # 28 x 28 (panjang x lebar)
hidden_size = 400 # jumlah neuron pada hidden layer
out_size = 10 # jumlah neuron pada output layer (0-9 : 10 digit)
batch_size = 100 # banyak sampel yang akan diproses dalam satu iterasi
epochs = 5
learning_rate = 0.001 # LR untuk optimizer

In [ ]:
# unduh dataset MNIST
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

  6%|▌         | 557k/9.91M [00:08<02:22, 65.6kB/s] 


KeyboardInterrupt: 